# Xây dựng hệ thống gợi ý sản phẩm sử dụng tập dữ liệu Amazon Reviews' 23

**Thành viên nhóm:**  
Nguyễn Hoàng Phúc - 22110400  
Phạm Trung Kỳ - 22110361  


## Thêm mục lục


## 1. Tóm tắt

Hệ thống gợi ý sản phẩm (recommender systems) là thành phần then chốt cho các nền tảng thương mại điện tử, giúp cá nhân hóa trải nghiệm người dùng và tăng tỷ lệ chuyển đổi mua hàng. Dự án này xây dựng một hệ thống gợi ý sản phẩm sử dụng dữ liệu đánh giá từ **Amazon Reviews 2023**, tập con **Arts, Crafts & Sewing** (tổng ~1.7M đánh giá). Nhóm thử nghiệm các phương pháp cổ điển và hiện đại: **Matrix Factorization** (SVD, ALS tối ưu bằng alternating least squares), tối ưu trực tiếp bằng **SGD**, **Neural Collaborative Filtering (NCF)** để học mối quan hệ phi tuyến giữa user & item, và **LSH** để tăng tốc tìm kiếm sản phẩm tương tự. Dữ liệu được tiền xử lý (loại bỏ bản ghi thiếu, đánh chỉ số User/Item, chuẩn hóa rating, xử lý văn bản cho mô hình text-aware). Hiệu năng được đánh giá bằng MAE, RMSE cho dự đoán rating và P@K, NDCG@K cho xếp hạng gợi ý. Kết quả thực nghiệm cho thấy ALS cho MAE/RMSE tốt hơn SVD; mô hình NCF và ALS cho kết quả tốt tương tự nhau. Báo cáo mô tả chi tiết thiết kế thí nghiệm, siêu tham số, kết quả, phân tích quá khớp, các bước tối ưu và đề xuất cải tiến trong tương lai.

## 2. Giới thiệu

**Bài toán.** Với tập đánh giá $\mathcal{D} = {(u, i, r_{ui})}$ (User ID, Item ID, Rating), mục tiêu là sẽ dự đoán rating cho các cặp $(u,i)$ chưa có và/hoặc sinh ra danh sách top-K sản phẩm cho mỗi user sao cho phù hợp với sở thích.

**Tại sao quan trọng.** Gợi ý cá nhân hóa giúp tăng trải nghiệm, tăng thời gian tương tác, và doanh thu. Bài toán thách thức do dữ liệu rất thưa (sparse), số lượng user/item lớn, và tính động của sở thích.

**Input / Output.**

* Input: ma trận đánh giá $R \in \mathbb{R}^{|U|\times|I|}$ (sparse), metadata (title, main category, description,...), review text.
* Output: (1) predicted rating $\hat r_{ui}$ cho cặp chưa xuất hiện, (2) danh sách các item top-K gợi ý cho user


## 3. Dữ liệu

Trong dự án này, nhóm sử dụng **Amazon Reviews Dataset (2023)** được công bố bởi McAuley Lab (UCSD). Đây là một trong những tập dữ liệu gợi ý sản phẩm quy mô lớn nhất hiện nay, với hơn 571,54 triệu đánh giá được thu thập từ nhiều danh mục hàng hóa khác nhau. Mỗi bản ghi (record) bao gồm thông tin đa dạng như:

* **User Reviews:** điểm đánh giá (rating), nội dung bình luận (review text), số lượt bình chọn “helpful”, v.v.
* **Item Metadata:** thông tin mô tả, giá, hình ảnh sản phẩm, danh mục.
* **Links:** dữ liệu quan hệ giữa sản phẩm và người dùng (ví dụ: “bought together”, “also viewed”).

Tập dữ liệu đã được nhóm tác giả chia sẵn thành ba phần train, validation, và test dựa trên **Absolute-Timestamp Splitting**. Cụ thể, mỗi người dùng có một chuỗi tương tác theo thời gian, và các mốc thời gian $t_1$ và $t_2$ được định nghĩa như sau:

* **Training part:** các tương tác có timestamp trong khoảng $(-\infty, t_1)$
* **Validation part:** các tương tác trong khoảng $[t_1, t_2)$
* **Testing part:** các tương tác trong khoảng $[t_2, +\infty)$

Với tập dữ liệu năm 2023, các giá trị được cố định là $t_1 = 1628643414042$ và $t_2 = 1658002729837$.
Cách chia này đảm bảo rằng dữ liệu mô phỏng thực tế, mô hình chỉ được huấn luyện trên dữ liệu quá khứ và đánh giá trên tương tác xảy ra sau đó, giúp kết quả phản ánh đúng tính chất dự đoán theo thời gian.


### 3.1. Dữ liệu sử dụng trong dự án

Nhóm chọn danh mục **“Arts, Crafts and Sewing”** làm tập chính để huấn luyện, với tổng cộng khoảng **1,8 triệu lượt đánh giá**, bao gồm:

* **197.3K người dùng (users)**
* **90.0K sản phẩm (items)**
* **Tỷ lệ chia tập:** 1.4M (train) / 192.8K (validation) / 210.8K (test)

Mỗi dòng dữ liệu bao gồm các trường chính sau:

<table>
  <thead>
    <tr>
      <th>Trường</th>
      <th>Mô tả</th>
    </tr>
  </thead>
  <tbody>
    <tr><td><code>user_id</code></td><td>Mã định danh người dùng</td></tr>
    <tr><td><code>parent_asin</code></td><td>Mã sản phẩm</td></tr>
    <tr><td><code>rating</code></td><td>Điểm đánh giá (1–5)</td></tr>
    <tr><td><code>timestamp</code></td><td>Thời điểm đánh giá</td></tr>
    <tr><td><code>history</code></td><td>Lịch sử các sản phẩm đã đánh giá trước đó</td></tr>
  </tbody>
</table>


Ví dụ về một số dòng dữ liệu trong tập train:

<table>
  <thead>
    <tr>
      <th>user_id</th>
      <th>parent_asin</th>
      <th>rating</th>
      <th>timestamp</th>
      <th>history</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>AFKZENTNBQ7A7V7UX...</td>
      <td>0913212148</td>
      <td>5.0</td>
      <td>1441260318000</td>
      <td>NULL</td>
    </tr>
    <tr>
      <td>AFKZENTNBQ7A7V7UX...</td>
      <td>B0BGXTQLL1</td>
      <td>5.0</td>
      <td>1523092443644</td>
      <td>0913212148</td>
    </tr>
    <tr>
      <td>AFKZENTNBQ7A7V7UX...</td>
      <td>B0BV5ZGRRM</td>
      <td>5.0</td>
      <td>1564347303691</td>
      <td>0913212148 B0BGXT...</td>
    </tr>
  </tbody>
</table>



Trong quá trình xử lý, nhóm chỉ sử dụng ba trường `user_idx`, `item_idx`, và `rating` cho các mô hình gợi ý chính (ALS, SGD, NCF, LSH). Riêng với **SVD**, do đặc thù thuật toán yêu cầu ma trận đánh giá đầy đủ, nhóm chuyển đổi dữ liệu sang dạng ma trận $R_{m \times n}$ (người dùng × sản phẩm).



### 3.2. Dữ liệu cho mô hình SVD

Ma trận đầy đủ được tạo thành để sử dụng cho thuật toán SVD là khá lớn và vì hạn chế phần cứng, nhóm sử dụng danh mục nhỏ hơn là **“All_Beauty”**, có kích thước khiêm tốn hơn nhiều:

* **253 người dùng**
* **356 sản phẩm**
* **Tập chia:** 2.2K (train) / 276 (validation) / 22 (test)

Danh mục này phù hợp để kiểm thử các thuật toán phân rã ma trận (Matrix Factorization) như **SVD** mà không gặp giới hạn về bộ nhớ hoặc thời gian huấn luyện.


### 3.3 Tiền xử lý dữ liệu

Trong dữ liệu gốc, `user_id` và `parent_asin` là kiểu chuỗi không tiện cho việc đưa vào mô hình, `rating` cũng chưa được chuẩn hóa dẫn đến việc huấn luyện lâu hơn, giá trị rating từ 0 đến 5 tiềm ẩn bias, khó khăn trong việc xử lý vấn đề cold-start.  
Mục tiêu là chuẩn hóa rating và tạo chỉ số user/item nhất quán giữa các split để phục vụ việc huấn luyện mô hình, xử lý vấn đề cold-start khi user/item mới xuất hiện từ tập valid hoặc test không có trong tập train.

  **Đọc dữ liệu**

   * Dữ liệu gồm ba split `train/valid/test` (giữ nguyên như mục 3.1).
   * Dữ liệu được đọc bằng PySpark để xử lý hiệu quả các tập dữ liệu lớn.
   * Ta xử lý riêng biệt user và item, nhưng chung quy trình.

  **Tạo mapping user/item**

   * Lấy tập hợp tất cả user/item xuất hiện trong các split, loại bỏ lặp lại, ta được tập chứa các user/item id.
   * Gán `userIndex` và `itemIndex` (unique integer) cho mỗi user/item, nhằm đảm bảo mapping nhất quán giữa các split và các lần chạy.
   * Lưu mapping ra file để tái sử dụng cho mô hình.

   **Chuẩn hóa rating theo item (item-mean normalization)**

   * Tính `item_mean = avg(rating)` trên train; tính `global_mean` trên train để dùng khi item mới xuất hiện trong valid/test.
   * Với mỗi split:

     * Gán index user/item theo mapping.
     * Gán `item_mean` cho mỗi record; nếu item chưa xuất hiện trong train → dùng `global_mean`.
     * Sinh cột chuẩn hóa: `rating_norm = rating - item_mean`.

   **Lưu dữ liệu tiền xử lý**

   * Lưu `item_means` và DataFrame đã chuẩn hóa (train/valid/test) để dùng trực tiếp cho mô hình.

   **Một số kết quả của quá trình xử lý dữ liệu**
   * User mapping

<table>
  <thead>
    <tr>
      <th style="text-align:left;">user_id</th>
      <th style="text-align:right;">userIndex</th>
    </tr>
  </thead>
  <tbody>
    <tr><td>AE22236AFRRSMQIKG...</td><td>0</td></tr>
    <tr><td>AE222H3FGXWLHRFUM...</td><td>1</td></tr>
    <tr><td>AE224QIIILW6WVFAE...</td><td>2</td></tr>
  </tbody>
</table>

  * Rating norm
<table>
  <tr>
    <th>itemIndex</th>
    <th>parent_asin</th>
    <th>userIndex</th>
    <th>user_id</th>
    <th>rating</th>
    <th>rating_norm</th>
    <th>item_mean</th>
  </tr>
  <tr>
    <td>89574</td><td>B0C7CXW1JB</td><td>193523</td><td>AHXOCRWNKC552ESFF...</td><td>4.0</td><td>-0.7849872773536894</td><td>4.7849872773536894</td>
  </tr>
  <tr>
    <td>21220</td><td>B00N9RIBIO</td><td>41477</td><td>AEV2AM5IEUVKSKLEJ...</td><td>5.0</td><td>0.5571428571428569</td><td>4.442857142857143</td>
  </tr>
  <tr>
    <td>89056</td><td>B0C4KB3GMD</td><td>193989</td><td>AHXY3L4R6MASA7JH4...</td><td>1.0</td><td>-3.0491803278688527</td><td>4.049180327868853</td>
  </tr>
  <tr>
    <td>75509</td><td>B09HBS73N9</td><td>183561</td><td>AHR7OPTZYKDBAAJQW...</td><td>5.0</td><td>0.4991011434207744</td><td>4.500898856579226</td>
  </tr>
  <tr>
    <td>77422</td><td>B09PFQ3NJW</td><td>970</td><td>AE2OBGULEEQ6SQPV6...</td><td>5.0</td><td>0.4991011434207744</td><td>4.500898856579226</td>
  </tr>
</table>




## 4. Phương pháp


### 4.1 Ý tưởng chung

Mỗi user $u$ và item $i$ được mô tả bằng vector đặc trưng $p_u, q_i \in \mathbb{R}^k$. Giá trị phù hợp (compatibility) được tính bằng dot product: $\hat r_{ui} = p_u^\top q_i$. Với NCF dùng mạng phi tuyến $f(p_u, q_i)$.  
Các phương pháp được sử dụng ở đây sẽ học các vector đặc trưng từ dữ liệu đánh giá của những người dùng với những item một cách tự động.


### 4.2 Matrix Factorization

Matrix Factorization là cách biểu diễn ma trận user-item **dưới dạng tích của hai ma trận nhỏ hơn**:

$$
R \approx P Q^\top
$$

* $R$ là ma trận rating gốc (user × item).
* $P$ là ma trận **user latent factors** (mỗi user thành một vector đặc trưng).
* $Q$ là ma trận **item latent factors** (mỗi item thành một vector đặc trưng).

Các thuật toán MF sẽ cố gắng tìm $P$ và $Q$ sao cho **tích $P Q^\top$ dự đoán rating gần đúng với rating thật**.


#### 4.2.1. SGD (Stochastic Gradient Descent) *

* Ý tưởng: **cập nhật từng user và item từng bước** dựa trên sai số dự đoán.
* Sai số dự đoán cho user $u$ với item $i$:

$$
e_{ui} = r_{ui} - p_u^\top q_i
$$

* Cập nhật vector latent với learning rate $\eta$:

$$
\begin{aligned}
p_u &\leftarrow p_u + \eta (e_{ui} q_i - \lambda p_u)\newline
q_i &\leftarrow q_i + \eta (e_{ui} p_u - \lambda q_i)
\end{aligned}
$$

* Giải thích:

  * $e_{ui} q_i$ và $e_{ui} p_u$ giúp giảm sai số dự đoán.
  * $\lambda p_u$ và $\lambda q_i$ là regularization để tránh overfitting.




#### 4.2.2. ALS (Alternating Least Squares)

* Ý tưởng là thay phiên nhau tối ưu $P$ rồi $Q$**.
* Khi $Q$ cố định, mỗi vector $p_u$ được tính bằng công thức **least squares**:

$$
p_u = (Q_{u}^\top Q_{u} + \lambda I)^{-1} Q_{u}^\top r_u
$$

* Trong đó:

  * $Q_u$ là ma trận chứa các $q_i$ mà user $u$ đã đánh giá.
  * $r_u$ là vector rating quan sát của user $u$.
* Tương tự, khi $P$ cố định, cập nhật $Q$.
* Ưu điểm là dễ song song hóa, xử lý ma trận thưa tốt.

#### 4.2.3. SVD (Singular Value Decomposition)

* Ý tưởng là tách ma trận rating $R \in \mathbb{R}^{m \times n}$ thành ba ma trận:
  $$
  R \approx U \Sigma V^\top
  $$

  * $U \in \mathbb{R}^{m \times k}$: đặc trưng người dùng
  * $\Sigma \in \mathbb{R}^{k \times k}$: giá trị kỳ dị
  * $V \in \mathbb{R}^{n \times k}$: đặc trưng sản phẩm

* **Truncated SVD:** chỉ giữ $k \ll \min(m, n)$ thành phần chính → giảm nhiễu, giảm chiều, tăng tốc.

* Kết quả ta được:
  $$
  \hat{R} = U \Sigma V^\top
  $$
  với $\hat{r}_{ui}$ là rating dự đoán; các hàng của $U, V$ là embedding của user và item.


### 4.3 Locality Sensitive Hashing (LSH)

Ý tưởng:

* LSH là kỹ thuật để tìm các vector gần nhau trong không gian lớn nhanh hơn so với tính toán tất cả cặp.
* Cách làm: ánh xạ mỗi vector vào một "bucket" sao cho các vector gần nhau thường rơi vào cùng bucket.
* Ứng dụng:

  * Tìm **nearest neighbors xấp xỉ** (approximate nearest neighbors).
  * Dùng cho item-based KNN hoặc tạo candidates trước khi xếp hạng bằng model chính.
  * Gợi ý top sản phẩm liên quan.

Cách hoạt động:

1. Chia không gian embedding bằng cách sinh ra một hoặc nhiều tập các mặt phẳng ngẫu nhiên (hyperplane).
2. Với mỗi vector:
   * Kiểm tra nó nằm phía nào của mỗi mặt phẳng (ví dụ: >0 hay <0).
   * Chuyển mỗi vector thành chuỗi nhị phân (0/1) → **hash code**.
3. Các vector có hash code giống nhau sẽ vào cùng bucket.
4. Khi vector mới vào sẽ được hash thành hashcode và thực hiện KNN với các vector trong bucket tương ứng.

### 4.4 Neural Collaborative Filtering (NCF)

Ý tưởng của việc sử dụng mạng neural network là để học mối quan hệ phi tuyến giữa các cặp vector embedding user $u$ và item $i$ trong việc dự đoán rating $\hat r_{ui}$.

**Input:**

* Mỗi user $u$ được ánh xạ thành vector embedding $p_u$
* Mỗi item $i$ được ánh xạ thành vector embedding $p_i$
* Các vector embedding này được khởi tạo ngẫu nhiên và sẽ được cải thiện dần khi mô hình học.

**Kết hợp embedding:**

* Ghép vector user và item thành một vector chung:
  $$
  x = [p_u, p_i]
  $$
* Vector này đại diện cho tương tác giữa user và item.

**MLP (Multi-Layer Perceptron):**

* Dữ liệu đi qua các lớp phi tuyến liên tiếp (Linear + Activation + Dropout/BatchNorm)
* MLP học cách ánh xạ từ vector embedding sang rating dự đoán, khám phá các tương tác phi tuyến phức tạp hơn giữa user và item.

**Output:**

* Lớp cuối cùng trả về **rating dự đoán** $\hat r_{ui}$
* Sử dụng **MSE loss** khi training để mạng học dự đoán rating thật với AdamW optimizer.






## 5. Thực nghiệm, kết quả và thảo luận

## 5.1 Thiết lập thí nghiệm




### A. Quy trình chung

* **Split**: train/valid/test giữ nguyên theo mục 3.1.
* **Metrics chính**:
  * Regression: **MAE**, **RMSE**:  
    $\mathrm{MAE} = \frac{1}{N}\sum |r_{ui}-\hat r_{ui}|$,  
    $\mathrm{RMSE} = \sqrt{\frac{1}{N}\sum (r_{ui}-\hat r_{ui})^2}$.  
  * Ranking: **Precision@K**, **NDCG@K**:  
    $\mathrm{DCG}_p = \sum_{i=1}^p \frac{rel_i}{\log_2(i+1)}$,  
    $\mathrm{NDCG}_p = \frac{\mathrm{DCG}_p}{\mathrm{IDCG}_p}$.  
    
* **Pipeline experiment**:

  1. Load dữ liệu đã tiền xử lý ở mục 3.3.
  2. Huấn luyện mô hình trên train, validation để chọn hyperparameter/early stopping, và cuối cùng đánh giá trên tập test.
  3. Lưu checkpoint, lịch sử training, kết quả experiment.

### B. ALS (Alternating Least Squares)

Nhóm sử dụng **ALS** trong Spark để huấn luyện mô hình phân rã ma trận, với mục tiêu tối ưu các siêu tham số `rank`, `regParam`, và `maxIter`. Quá trình tìm kiếm được thực hiện bằng **grid search** trên các giá trị: `rank = 10`, `regParam ∈ {0.01, 0.1, 1.0}`, và `maxIter ∈ {5, 10, 15}`. Mỗi tổ hợp được huấn luyện trên tập train và đánh giá trên tập validation bằng các độ đo RMSE và MAE.

Kết quả cho thấy cấu hình **rank = 10, regParam = 1.0, maxIter = 10–15** đạt hiệu năng tốt nhất, với **RMSE=1.1078** và **MAE=0.7635** cân bằng giữa train và validation, cho thấy việc tăng giá trị regularization giúp giảm hiện tượng overfitting rõ rệt so với các giá trị nhỏ hơn. Mô hình cũng đạt **0.9821 NDCG@10** và **0.7952 Precision@10** trên tập test



<img src="images/als_train_rmse_heatmap.png" alt="ALS Train RMSE Heatmap" width="650">  

**Hình 1.** Phân bố **RMSE trên tập huấn luyện** của mô hình ALS theo các giá trị siêu tham số.  
_Biểu đồ heatmap thể hiện sai số RMSE tương ứng với từng cặp (maxIter, hệ số regularization λ), cho thấy mức độ hội tụ và khả năng khớp dữ liệu huấn luyện của mô hình._


<img src="images/als_valid_rmse_heatmap.png" alt="ALS Validation RMSE Heatmap" width="650">  

**Hình 2.** Phân bố **RMSE trên tập kiểm định** của mô hình ALS theo các giá trị siêu tham số.  
_Biểu đồ cho thấy ảnh hưởng của regularization và số vòng lặp đến khả năng tổng quát hóa của mô hình; λ lớn giúp giảm sai số kiểm định và hạn chế overfitting._


<img src="images/als_train_valid_rmse_vs_iter.png" alt="ALS Train vs Validation RMSE" width="650">  

**Hình 3.** So sánh **RMSE giữa tập huấn luyện và tập kiểm định** theo số vòng lặp `maxIter` và các giá trị regularization λ.  
_Biểu đồ lineplot cho thấy xu hướng hội tụ: RMSE tập huấn luyện giảm rất nhẹ khi tăng số vòng lặp, trong khi RMSE tập kiểm định ổn định hơn với regularization cao giảm mạnh overfitting, thể hiện khả năng tổng quát hóa của mô hình._




<img src="images/als_train_valid_test_metrics.png" alt="ALS Train, Validation, Test Metrics" width="650">  

**Hình 4.** So sánh **RMSE và MAE trên các tập Train, Validation và Test**.  
_Biểu đồ barplot tổng hợp giúp đánh giá hiệu năng chung của mô hình trên tất cả các tập dữ liệu, đồng thời quan sát sự cân bằng giữa huấn luyện và kiểm định, cũng như khả năng dự đoán trên tập Test._

Khi regParam nhỏ (0.01–0.1), mô hình đạt sai số huấn luyện rất thấp nhưng sai số kiểm định cao, cho thấy hiện tượng overfitting rõ rệt. Khi tăng regParam lên 1.0, sai số huấn luyện tăng nhẹ trong khi sai số kiểm định giảm và ổn định hơn, chứng tỏ mô hình tổng quát hóa tốt hơn. Ngoài ra, việc tăng maxIter giúp mô hình hội tụ tốt hơn ở các giá trị nhỏ, nhưng ảnh hưởng không đáng kể khi regParam cao. Nhìn chung, cấu hình rank = 10, regParam = 1.0, maxIter = 10–15 cho kết quả cân bằng và đáng tin cậy nhất.

<img src="images/rating_distribution.png" alt="Rating Distribution" width="650">  


**Hình 5.** Phân phối điểm đánh giá (rating distribution) trong tập dữ liệu.  
_Biểu đồ histogram thể hiện tần suất xuất hiện của từng mức rating từ 1 đến 5._


Phân phối điểm đánh giá bị lệch phải, trong đó rating 4 và 5 chiếm tỷ lệ lớn nhất, tiếp đến là rating 1, 3 và 2 thấp hơn nhiều. Mô hình dự đoán tập trung chủ yếu trong khoảng 4.5–5, đạt đỉnh ở 4.5, cho thấy mô hình có xu hướng định giá các sản phẩm ở mức cao, phản ánh dữ liệu huấn luyện bị thiên lệch về các đánh giá tích cực. Điều này nhấn mạnh tầm quan trọng của việc sử dụng regularization hoặc normalization để giảm sai lệch trong dự đoán và cải thiện khả năng tổng quát hóa.


### C. SGD (Stochastic Gradient Descent)


Mô hình phân rã ma trận được huấn luyện bằng thuật toán SGD nhằm tối thiểu hóa sai số dự đoán giữa người dùng và sản phẩm. Nhóm thử nghiệm hai cấu hình siêu tham số khác nhau để đánh giá ảnh hưởng của số chiều ẩn $K$ và tốc độ học. Các tham số cụ thể gồm: $K = {30, 50}$, learning rate $lr = {0.007, 0.01}$, hệ số regularization $\lambda = {0.02, 0.05}$, số epoch = 100, và patience = 20 cho early stopping. Ma trận ẩn $P, Q$ được khởi tạo ngẫu nhiên trong khoảng $[0, 0.1]$.

Kết quả được đánh giá bằng các độ đo RMSE, Precision@K, và NDCG@K trên tập validation và test. Cấu hình tốt nhất đạt **MAE_test = 0.704**, **RMSE_test = 1.0948**, **Precision@K = 0.8878**, và **NDCG@K = 0.9819** khi $K=50$, $lr=0.01$, $\lambda=0.05$. Train loss giảm nhanh trong các epoch đầu, trong khi validation loss thay đổi rất ít, cho thấy dữ liệu khá đơn giản và mô hình hội tụ sớm.





<img src="images/sgd_rmse_mae_train_valid_loss.png" alt="Training and Validation Loss of SGD" width="650">  


**Hình 6.** Biểu đồ huấn luyện mô hình SGD – train loss và validation loss theo số epoch.


Về RMSE thì biểu đồ cho thấy train loss giảm nhanh, trong khi validation loss hầu như không giảm đáng kể. Điều này cho thấy dữ liệu tương đối đơn giản, khiến mô hình hội tụ sớm và có overfitting rõ rệt. Tuy nhiên, khoảng cách nhỏ giữa RMSE train và validation loss cũng gợi ý rằng mô hình đã gần đạt giới hạn hiệu năng với cấu hình siêu tham số hiện tại. Với MAE loss thì ngoài việc có giá trị thấp hơn RMSE thì cũng cho một hình dáng đồng nhất.



### D. NCF (Neural Collaborative Filtering)

Nhóm huấn luyện mô hình NCF nhằm học biểu diễn phi tuyến giữa người dùng và sản phẩm thông qua mạng nơ-ron. Hai cấu hình siêu tham số được thử nghiệm với tốc độ học ban đầu $lr = {0.001, 0.0001}$, cùng cơ chế dừng sớm (early stopping) sau 24 và 27 epoch. Các tham số còn lại được giữ cố định, với kiến trúc mạng gồm các tầng fully-connected và hàm kích hoạt ReLU.

Kết quả cho thấy mô hình đạt **MAE_test = 0.7382**, **RMSE_test = 1.0878**, **Precision@K = 0.7952**, và **NDCG@K = 0.982** khi $lr = 0.001$. Biểu đồ training loss cho thấy mô hình hội tụ sớm, cả training và validation loss đều gần như không giảm, với validation loss cao hơn, cho thấy NCF khó tối ưu hơn và có thể đang rơi vào vùng mất cân bằng giữa underfitting và hội tụ sớm.




<img src="images/ncf_rmse_mae_train_valid_loss.png" alt="Training and Validation Loss of NCF" width="650">

**Hình 7.** Biểu đồ huấn luyện mô hình NCF – train loss và validation loss theo số epoch.

Biểu đồ cho thấy RMSE và MAE loss có xu hướng giống nhau, cả training loss và validation loss gần như giữ nguyên trong suốt quá trình huấn luyện, không giảm đáng kể và mô hình dừng sớm sau khoảng gần 35 epoch. Hiện tượng này cho thấy mô hình hội tụ sớm nhưng chưa học được nhiều từ dữ liệu, có khả năng rơi vào trạng thái underfitting. Tuy nhiên Validation loss luôn cao hơn train loss, nhưng khoảng cách không phải lớn và có thể cần điều chỉnh kiến trúc mạng hoặc tốc độ học để xem cải thiện hiệu năng tốt hơn nữa được hay không.


### D. SVD

* **ALS:** grid search trên `rank`, `regParam`, `maxIter`.
* **NCF:** thử nghiệm các tổ hợp `embedding_dim`, `lr`, `dropout`, `batch_size`.
* Lưu mọi kết quả experiment, logs, checkpoint để reproducibility.

### F. LSH (Locality-Sensitive Hashing)

Để tăng tốc việc tìm kiếm các item tương tự dựa trên vector embedding (user hoặc item), nhóm áp dụng **BucketedRandomProjectionLSH** với `bucketLength = 2.0` và `numHashTables = 3`. Tập dữ liệu gồm ~79k item vector 10 chiều, kết hợp metadata như `title`, `main_category` và `store`.

Mô hình được huấn luyện trên toàn bộ tập vector, sau đó thực hiện truy vấn top-5 items gần nhất cho các vector user và item. Kết quả cho thấy LSH trả về các vector rất gần với truy vấn (Euclidean distance gần 0) trong thời gian trung bình ~0.91–1.08 giây, phù hợp với các ứng dụng gợi ý trực tuyến.

Cấu hình hiện tại cân bằng tốt giữa độ chính xác và tốc độ truy vấn, đồng thời giúp tránh tính toán brute-force trên toàn bộ tập dữ liệu lớn.


### 5.2 Kết quả

Bảng 5.2 tổng hợp hiệu năng của các mô hình gợi ý sản phẩm trên tập dữ liệu *Arts_Crafts_and_Sewing*, bao gồm MAE, RMSE, Precision@10, NDCG@10 và **Overall Score** — chỉ số tổng hợp đánh giá toàn diện khả năng dự đoán và xếp hạng của mô hình.

<table>
  <thead>
    <tr>
      <th>Mô hình</th>
      <th>MAE</th>
      <th>RMSE</th>
      <th>Precision@10</th>
      <th>NDCG@10</th>
      <th>Overall Score</th>
    </tr>
  </thead>
  <tbody>
    <tr><td>SVD</td><td>0.7694</td><td>1.0634</td><td>0.2064</td><td>0.9651</td><td>0.664</td></tr>
    <tr><td>SGD</td><td>0.7040</td><td>1.0948</td><td>0.8878</td><td>0.9819</td><td>0.882</td></tr>
    <tr><td>ALS</td><td>0.7635</td><td>1.1078</td><td>0.7952</td><td>0.9821</td><td>0.855</td></tr>
    <tr><td>NCF</td><td>0.7382</td><td>1.0878</td><td>0.7952</td><td>0.9820</td><td>0.861</td></tr>
  </tbody>
</table>



Công thức tính Overall Score:

$$
  \text{Overall Score} = 0.4 \times \text{RMSE\_norm} + 0.4 \times \text{Precision@10} + 0.2 \times \text{NDCG@10}
$$

$$
  \text{RMSE\_norm} = 1 - \frac{\text{RMSE} - \min(\text{RMSE})}{\max(\text{RMSE}) - \min(\text{RMSE})}
$$

Trong đó, RMSE_norm được đảo lại để giá trị RMSE thấp tương ứng với score cao hơn. Overall Score kết hợp cả khả năng dự đoán chính xác rating và khả năng xếp hạng top-K items, cho phép so sánh tổng thể giữa các mô hình.

Từ bảng trên, có thể thấy **SGD đạt Overall Score cao nhất (0.882)**, nhờ kết hợp tốt giữa độ chính xác rating (MAE thấp) và khả năng xếp hạng top-K (Precision@10 cao), trong khi **SVD** có Overall Score thấp nhất (0.664) do Precision@10 kém. **ALS** và **NCF** thể hiện hiệu năng ổn định, với Overall Score lần lượt là 0.855 và 0.861.

<img src="images/model_comparison.png" alt="Model Comparison" width="650">

**Hình 8.** So sánh hiệu năng giữa các mô hình gợi ý sản phẩm.  
_Biểu đồ thể hiện giá trị NDCG@10 và Precision@10 của các mô hình SVD, SGD, ALS, và NCF trên tập dữ liệu Arts_Crafts_and_Sewing._
 
Mô hình SGD đạt Precision@10 cao nhất và NDCG@10 gần cao nhất, cho thấy khả năng xếp hạng top-K items chính xác hơn so với các mô hình khác. ALS và NCF có hiệu năng ổn định ở cả hai thước đo, với Precision@10 và NDCG@10 gần như bằng nhau, cho thấy cả hai mô hình đều mạnh trong việc duy trì ranking tổng thể. Trong khi đó, SVD có Precision@10 thấp nhất mặc dù NDCG@10 vẫn tương đối cao, chỉ ra rằng mô hình này bị ảnh hưởng rất nhiều bởi nhiễu do ma trận thưa, khó phân biệt các item hàng đầu nhưng vẫn giữ được ranking chung tương đối.





## 6. Kết luận

Nhóm đã triển khai và so sánh nhiều phương pháp gợi ý trên tập dữ liệu Amazon Arts & Crafts: SVD, SGD, ALS và NCF. Kết quả cho thấy:

* **SGD** nổi bật về độ chính xác dự đoán (MAE thấp nhất), phù hợp nếu mục tiêu là dự đoán rating.
* **ALS** và **NCF** mạnh về ranking top-K (Precision@10 và NDCG@10 cao nhất), phù hợp nếu mục tiêu là gợi ý các sản phẩm hàng đầu cho người dùng.
* **SVD** thể hiện kém hơn về ranking, cho thấy các mô hình tuyến tính đơn giản khó cạnh tranh với các mô hình phi tuyến hoặc tối ưu trực tiếp cho ranking.

Hướng phát triển tiếp theo có thể bao gồm:

1. Fine-tune embedding người dùng và sản phẩm, tích hợp thêm thông tin meta-data.
2. Kết hợp hai bước (candidate generation + ranking) để cải thiện hiệu năng top-K.
3. Triển khai A/B test thực tế nếu có dữ liệu người dùng thật để đánh giá cuối cùng.


---

## 7. Phụ lục

### A. Chuẩn hóa dữ liệu theo item mean

Gọi:

* $r_{ui}$ là rating gốc của user $u$ cho item $i$,
* $I_u, U_i$ là tập các user và item tương ứng,
* $ \mu_i $ là rating trung bình của item $i$ trên tập train:
  $$
  \mu_i = \frac{1}{|U_i|} \sum_{u \in U_i} r_{ui}.
  $$

Sau đó, rating được chuẩn hóa thành:
$$
r^{\text{norm}}*{ui} = r*{ui} - \mu_i.
$$

* Nếu item mới xuất hiện trong tập validation hoặc test mà không có giá trị trung bình từ train, nhóm sử dụng **global mean** $\mu_{\text{global}}$:
  $$
  r^{\text{norm}}*{ui} =
  \begin{cases}
  r*{ui} - \mu_i, & \text{nếu } i \in \text{train items},\newline
  r_{ui} - \mu_{\text{global}}, & \text{nếu } i \notin \text{train items}.
  \end{cases}
  $$



### B. Công thức cập nhật trong SGD

Ta tối thiểu hóa hàm mất mát (cho toàn bộ dữ liệu quan sát được):
$$
\mathcal{L} = \sum_{(u,i)\in\mathcal{D}} \frac{1}{2}(r_{ui} - p_u^\top q_i)^2 + \frac{\lambda}{2}\big(|p_u|^2 + |q_i|^2\big)
$$
đặt sai số (prediction error) cho một cặp $(u,i)$ là
$$
e_{ui} = r_{ui} - p_u^\top q_i.
$$

Với chuẩn viết có hệ số $1/2$ phía trước bình phương, gradient theo $p_u$ và $q_i$ của thành phần mất mát tại cặp $(u,i)$ là:
$$
\begin{aligned}
\frac{\partial}{\partial p_u}\Big(\tfrac{1}{2} e_{ui}^2 + \tfrac{\lambda}{2}|p_u|^2\Big)
&= - e_{ui} q_i + \lambda p_u,\newline
\frac{\partial}{\partial q_i}\Big(\tfrac{1}{2} e_{ui}^2 + \tfrac{\lambda}{2}|q_i|^2\Big)
&= - e_{ui} p_u + \lambda q_i.
\end{aligned}
$$

Áp dụng quy tắc cập nhật SGD với learning rate (\eta) (cập nhật theo hướng ngược gradient):
$$
\begin{aligned}
p_u &\leftarrow p_u - \eta\Big(- e_{ui} q_i + \lambda p_u\Big)
= p_u + \eta\big(e_{ui} q_i - \lambda p_u\big),\newline
q_i &\leftarrow q_i - \eta\Big(- e_{ui} p_u + \lambda q_i\Big)
= q_i + \eta\big(e_{ui} p_u - \lambda q_i\big).
\end{aligned}
$$

**Ghi chú về các dạng khác nhau của công thức:** Một số hàm mất mát không có hệ số $1/2$ (ví dụ $(r-\hat r)^2 + \lambda|.|^2$), thì gradient sẽ chứa nhân tố $2$ và cập nhật sẽ có thêm hệ số $2$. Người thực nghiệm thường gộp hệ số $2$ vào $\eta$ (hoặc dùng biểu thức có $1/2$ như trên) để có dạng cập nhật đơn giản như trình bày (nhưng cuối cùng về mục tiêu tối ưu vẫn như nhau).



### C. Công thức cập nhật trong ALS (Alternating Least Squares)

Ý tưởng là tách bài toán tối ưu đồng thời theo $P$ và $Q$ thành hai bước lặp: cố định $Q$ rồi tối ưu $P$ (từng hàng $p_u$ một), sau đó cố định $P$ và tối ưu $Q$. Với định nghĩa mất mát toàn cục
$$
\mathcal{L} = \sum_{(u,i)\in\mathcal{D}} (r_{ui} - p_u^\top q_i)^2 + \lambda\Big(\sum_u |p_u|^2 + \sum_i |q_i|^2\Big),
$$
xét một user cụ thể $u$. Gọi $I_u$ là tập các item mà user $u$ đã đánh giá; xây hai đối tượng:

* $r_u\in\mathbb{R}^{|I_u|}$: vector giá trị rating quan sát (thứ tự tương ứng với $I_u$);
* $Q_u\in\mathbb{R}^{|I_u|\times K}$: ma trận có các hàng là $q_i^\top$ với $i\in I_u$.

Khi cố định mọi $q_i$, bài toán đối với $p_u$ là:
$$
\min_{p_u} ||r_u - Q_u p_u||^2 + \lambda ||p_u||^2.
$$
Đây là bài toán least squares đã có công thức giải, công thức có dạng:
$$
(Q_u^\top Q_u + \lambda I_K) p_u = Q_u^\top r_u.
$$

$$
p_u = (Q_u^\top Q_u + \lambda I)^{-1} Q_u^\top r_u,
$$

Trong thực thi, $Q_u^\top Q_u = \sum_{i\in I_u} q_i q_i^\top$ (ma trận $K\times K$) và $Q_u^\top r_u = \sum_{i\in I_u} r_{ui}, q_i$ (vector kích thước $K$).  
Tương tự, khi cố định $P$ ta có:
$$
q_i = (P_i^\top P_i + \lambda I)^{-1} P_i^\top r_i
$$
với $P_i$ ma trận gồm các $p_u^\top$ cho user đã đánh giá item $i$, và $r_i$ là vector rating quan sát cho item $i$.


## 8. Đóng góp (Contributions)

* **Nguyễn Hoàng Phúc:** tiền xử lý dữ liệu, triển khai ALS, triển khai LSH, soạn và biên tập báo cáo.
* **Phạm Trung Kỳ:** triển khai SVD, SGD, NCF, thực hiện thí nghiệm và soạn báo cáo.


## 9. Tham khảo (References)


* Deisenroth, M. P., Faisal, A. A., & Ong, C. S. (2020). *Mathematics for Machine Learning*. Cambridge University Press. pp. 111–134.

* GeeksforGeeks. (2025, September 30). Stochastic Gradient Descent. Retrieved from [https://www.geeksforgeeks.org/machine-learning/ml-stochastic-gradient-descent-sgd/](https://www.geeksforgeeks.org/machine-learning/ml-stochastic-gradient-descent-sgd/)

* CodeSignal. (2025). Implementing the Alternating Least Squares Algorithm. Retrieved from [https://codesignal.com/learn/courses/diving-deep-into-collaborative-filtering-techniques-with-als/lessons/implementing-the-alternating-least-squares-algorithm](https://codesignal.com/learn/courses/diving-deep-into-collaborative-filtering-techniques-with-als/lessons/implementing-the-alternating-least-squares-algorithm)

* Pinecone. (2025). Locality Sensitive Hashing (LSH): The Illustrated Guide. Retrieved from [https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing/](https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing/)

* GeeksforGeeks. (2025, July 23). Overfitting and Regularization in ML. Retrieved from [https://www.geeksforgeeks.org/machine-learning/overfitting-and-regularization-in-ml/](https://www.geeksforgeeks.org/machine-learning/overfitting-and-regularization-in-ml/)

* Apache Spark. (2025). Machine Learning Library (MLlib) Guide. Retrieved from [https://spark.apache.org/docs/latest/ml-guide.html](https://spark.apache.org/docs/latest/ml-guide.html)
